## Day 9: Stream Processing

A large stream blocks your path. According to the locals, it's not safe to cross the stream at the moment because it's full of garbage. You look down at the stream; rather than water, you discover that it's a stream of characters.

You sit for a while and record part of the stream (your puzzle input). The characters represent groups - sequences that begin with { and end with }. Within a group, there are zero or more other things, separated by commas: either another group or garbage. Since groups can contain other groups, a } only closes the most-recently-opened unclosed group - that is, they are nestable. Your puzzle input represents a single, large group which itself contains many smaller ones.

Sometimes, instead of a group, you will find garbage. Garbage begins with < and ends with >. Between those angle brackets, almost any character can appear, including { and }. Within garbage, < has no special meaning.

In a futile attempt to clean up the garbage, some program has canceled some of the characters within it using !: inside garbage, any character that comes after ! should be ignored, including <, >, and even another !.

You don't see any characters that deviate from these rules. Outside garbage, you only find well-formed groups, and garbage always terminates according to the rules above.

Here are some self-contained pieces of garbage:

    <>, empty garbage.
    <random characters>, garbage containing random characters.
    <<<<>, because the extra < are ignored.
    <{!>}>, because the first > is canceled.
    <!!>, because the second ! is canceled, allowing the > to terminate the garbage.
    <!!!>>, because the second ! and the first > are canceled.
    <{o"i!a,<{i<a>, which ends at the first >.

Here are some examples of whole streams and the number of groups they contain:

    {}, 1 group.
    {{{}}}, 3 groups.
    {{},{}}, also 3 groups.
    {{{},{},{{}}}}, 6 groups.
    {<{},{},{{}}>}, 1 group (which itself contains garbage).
    {<a>,<a>,<a>,<a>}, 1 group.
    {{<a>},{<a>},{<a>},{<a>}}, 5 groups.
    {{<!>},{<!>},{<!>},{<a>}}, 2 groups (since all but the last > are canceled).

Your goal is to find the total score for all groups in your input. Each group is assigned a score which is one more than the score of the group that immediately contains it. (The outermost group gets a score of 1.)

    {}, score of 1.
    {{{}}}, score of 1 + 2 + 3 = 6.
    {{},{}}, score of 1 + 2 + 2 = 5.
    {{{},{},{{}}}}, score of 1 + 2 + 3 + 3 + 3 + 4 = 16.
    {<a>,<a>,<a>,<a>}, score of 1.
    {{<ab>},{<ab>},{<ab>},{<ab>}}, score of 1 + 2 + 2 + 2 + 2 = 9.
    {{<!!>},{<!!>},{<!!>},{<!!>}}, score of 1 + 2 + 2 + 2 + 2 = 9.
    {{<a!>},{<a!>},{<a!>},{<ab>}}, score of 1 + 2 = 3.

What is the total score for all groups in your input?

In [1]:
testcases = (
    ('{}', 1),                                                          
    ('{{{}}}', 6),                                                      
    ('{{},{}}', 5),                                                     
    ('{{{},{},{{}}}}', 16),                                             
    ('{<a>,<a>,<a>,<a>}', 1),                                           
    ('{{<ab>},{<ab>},{<ab>},{<ab>}}', 9),                               
    ('{{<!!>},{<!!>},{<!!>},{<!!>}}', 9),                               
    ('{{<a!>},{<a!>},{<a!>},{<ab>}}', 3),
)

In [2]:
def parse_garbage(stream):
    for char in stream:
        if char == '!':
            x = next(stream)
            continue
        
        if char == '>':
            return

def parse_group(stream, level=0):
    score = 1 * level
    
    for char in stream:
        if char == '{':
            score += parse_group(stream, level + 1)
        elif char == '}':
            return score
        elif char == '<':
            parse_garbage(stream)
    
    return score

def chars_from_file(filename):
    with open(filename) as data:
        for line in data:
            for char in line:
                yield char

In [3]:
def run_tests(test_func, cases):
    for case, expected in cases:
        print("Testing", repr(case))
        actual = test_func(case)
    
        if actual != expected:
            print("Failed:", actual, '!=', expected)
        else:
            print("OK:", actual, '==', expected)

In [4]:
run_tests(lambda case: parse_group(iter(case)), testcases)

Testing '{}'
OK: 1 == 1
Testing '{{{}}}'
OK: 6 == 6
Testing '{{},{}}'
OK: 5 == 5
Testing '{{{},{},{{}}}}'
OK: 16 == 16
Testing '{<a>,<a>,<a>,<a>}'
OK: 1 == 1
Testing '{{<ab>},{<ab>},{<ab>},{<ab>}}'
OK: 9 == 9
Testing '{{<!!>},{<!!>},{<!!>},{<!!>}}'
OK: 9 == 9
Testing '{{<a!>},{<a!>},{<a!>},{<ab>}}'
OK: 3 == 3


In [5]:
parse_group(chars_from_file('day9.txt'))

11347

## Part Two

Now, you're ready to remove the garbage.

To prove you've removed it, you need to count all of the characters within the garbage. The leading and trailing < and > don't count, nor do any canceled characters or the ! doing the canceling.

    <>, 0 characters.
    <random characters>, 17 characters.
    <<<<>, 3 characters.
    <{!>}>, 2 characters.
    <!!>, 0 characters.
    <!!!>>, 0 characters.
    <{o"i!a,<{i<a>, 10 characters.

How many non-canceled characters are within the garbage in your puzzle input?

In [6]:
garbage_tests = (
    ('<>', 0),
    ('<random characters>', 17),
    ('<<<<>', 3),
    ('<{!>}>', 2),
    ('<!!>', 0),
    ('<!!!>>', 0),
    ('<{o"i!a,<{i<a>', 10),
)

In [7]:
def parse_garbage(stream):
    count = 0
    
    for char in stream:
        if char == '!':
            x = next(stream)
            continue
        
        if char == '>':
            return count
        
        count += 1

def parse_group(stream, level=0):
    score = 1 * level
    garbage = 0
    
    for char in stream:
        if char == '{':
            nscore, ngarbage = parse_group(stream, level + 1)
            score += nscore
            garbage += ngarbage
        elif char == '}':
            return score, garbage
        elif char == '<':
            garbage += parse_garbage(stream)
    
    return score, garbage

In [8]:
run_tests(lambda case: parse_group(iter(case))[1], garbage_tests)

Testing '<>'
OK: 0 == 0
Testing '<random characters>'
OK: 17 == 17
Testing '<<<<>'
OK: 3 == 3
Testing '<{!>}>'
OK: 2 == 2
Testing '<!!>'
OK: 0 == 0
Testing '<!!!>>'
OK: 0 == 0
Testing '<{o"i!a,<{i<a>'
OK: 10 == 10


In [9]:
parse_group(chars_from_file('day9.txt'))[1]

5404